In [1]:
import pandas as pd
import sklearn as ktl

In [41]:
bol = pd.read_csv("/Users/hendrixperalta/Desktop/bolivia/data/GeoDS4Bolivia.csv")
bol = bol.rename(columns={"asdf_id":"id"})
#list(bol.columns)
sat = pd.read_csv("data/egdp/satelite_data.csv")
df = pd.merge(bol, sat, on="id", how="outer")


In [28]:
sat

,id,egdp1992,egdp1993,egdp1994,egdp1995,egdp1996,egdp1997,egdp1998,egdp1999,egdp2000,...,urb_land2013,urb_land2014,urb_land2015,urb_land2016,urb_land2017,urb_land2018,urb_land2019,urb_land2020,urb_land2021,urb_land2022
0,0,9.841833,10.120295,10.433873,10.757316,11.068601,11.432251,11.813545,11.846990,12.044015,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,15.909156,16.359285,16.866179,17.389018,17.892206,18.480040,19.096395,19.150457,19.468946,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,17.378421,17.870121,18.423828,18.994953,19.544612,20.186735,20.860013,20.919068,21.266969,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,29.115081,31.702251,32.941582,34.683048,36.866041,38.260194,39.590650,39.908588,41.190774,...,355.160182,367.906971,413.543046,486.786762,462.030631,538.099996,515.804994,444.434997,455.805004,464.310006
4,4,8.896404,9.148116,9.431571,9.723943,10.005325,10.334042,10.678708,10.708940,10.887039,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,334,40.622615,41.771979,43.066288,44.401310,45.686155,47.187138,48.766946,48.905006,49.718337,...,1.254882,4.644307,5.030808,4.436374,4.895093,4.950000,9.515000,8.480000,8.480000,7.770000
335,335,110.382107,113.505227,117.022196,120.649852,124.141110,128.221110,132.497604,132.872706,135.082490,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
336,336,36.579590,37.621999,38.940809,40.165984,41.401073,42.768805,44.195252,44.349411,45.127559,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
337,337,79.350290,86.806697,92.473700,95.915445,99.559252,103.230407,106.516053,106.295897,107.656011,...,322.223933,386.937917,451.182560,495.108385,590.187676,678.539684,726.000970,660.810332,674.485428,651.174429


In [32]:
x = df[["ln_NTLpc2012", "ln_t400NTLpc2012", "pop2012", 'egdp2012', 'urb_land2012']].fillna(0)
y = df['imds']

In [33]:
list(df.columns)

['poly_id',
 'id',
 'mun',
 'mun_id',
 'dep',
 'dep_id',
 'dep_mun',
 'shapeID',
 'imds',
 'rank_imds',
 'population_2020',
 'urbano_2012',
 'sdg1_1_eepr',
 'sdg1_1_ubn',
 'sdg1_2_mpi',
 'sdg1_4_abs',
 'sdg2_2_cmc',
 'sdg2_2_oww',
 'sdg2_4_pual',
 'sdg2_4_td',
 'sdg3_1_idca',
 'sdg3_2_imr',
 'sdg3_2_mrc',
 'sdg3_3_cdir',
 'sdg3_3_di',
 'sdg3_3_imr',
 'sdg3_3_ti',
 'sdg3_3_hivi',
 'sdg3_7_afr',
 'sdg4_1_ssdrm',
 'sdg4_1_ssdrf',
 'sdg4_4_phe',
 'sdg4_6_lr',
 'sdg4_c_qti',
 'sdg4_c_qts',
 'sdg5_1_gpsd',
 'sdg5_1_gpyp',
 'sdg5_1_gpmpi',
 'sdg5_5_gpop',
 'sdg6_1_dwc',
 'sdg6_2_sc',
 'sdg6_3_wwt',
 'sdg7_1_ec',
 'sdg7_1_rec',
 'sdg7_1_cce',
 'sdg7_3_co2epc',
 'sdg8_4_rem',
 'sdg8_5_oprm',
 'sdg8_5_ofrm',
 'sdg8_6_mlm',
 'sdg8_6_wlm',
 'sdg8_10_dbb',
 'sdg8_11_idi',
 'sdg9_1_routes',
 'sdg9_5_cd',
 'sdg9_5_eutf',
 'sdg9_c_mnc',
 'sdg9_c_drb',
 'sdg10_2_gcye',
 'sdg10_2_iec',
 'sdg10_2_nssp',
 'sdg11_1_hocr',
 'sdg11_1_hno',
 'sdg11_2_samt',
 'sdg13_1_ccvi',
 'sdg13_2_tco2e',
 'sdg13_2_dra',
 

In [34]:
y.dtypes

dtype('float64')

In [35]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.get_params()



{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [36]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3) 

In [37]:
#clf.fit(x_train,y_train)

In [38]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [39]:
model.fit(x_train,y_train)

LinearRegression()

In [40]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 29.704815600124032
R^2 Score: 0.3756418692497624
